In [1]:
# Useful for defining quantities
from astropy import units as u

# Earth focused modules, ISS example orbit and time span generator
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.twobody.orbit import Orbit
from poliastro.bodies import Earth
from astropy.time import Time
from poliastro.examples import iss
from poliastro.util import time_range
from TLE_OP_function import *

In [2]:
constellation = (
    "FOSSASAT-2E2", 
    # "FOSSASAT-2E1",
    # "FOSSASAT-2E4", 
    # "FOSSASAT-2E3", 
    # "FOSSASAT-2E6", 
    # "FOSSASAT-2E5", 
    # "FOSSASAT-2E11",	
    # "FOSSASAT-2E12",	
    # "FOSSASAT-2E13",	
    # "FOSSASAT-2E7", 
    # "FOSSASAT-2E8" 
)

# Obtengo los parámetros orbitales para la última actualización del TLE del satélite
RAAN_deg, i_deg, e, a, AOP_deg, MA_deg = from_TLE_to_OrbParams(constellation)

sat = list(load_gp_from_celestrak(name=constellation[0]))[0]
sat_dict = sat_to_dict(sat,constellation[0])
time = Time(sat_dict["EPOCH"], scale='utc')

In [3]:
# Build spacecraft instance
a = a[0] * u.km
ecc = e[0] * u.one
inc = i_deg[0] * u.deg
raan = RAAN_deg[0] * u.deg
argp = AOP_deg[0] * u.deg
nu = 0 * u.deg

print (a, ecc, inc, raan, argp, nu)
FOSSA_epoch = Time(sat_dict["EPOCH"], format='isot', scale='utc')
FOSSA_epoch.format = 'iso'
print(FOSSA_epoch)
FOSSA_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, epoch=FOSSA_epoch)
FOSSA_spacecraft = EarthSatellite(FOSSA_orbit, None)
t_span = time_range(
    FOSSA_epoch, periods=300, end=FOSSA_epoch + 3600 * u.s
)

6858.891007971385 km 0.0011503 97.4694 deg 67.5383 deg 99.5516 deg 0.0 deg
2022-12-26 18:49:13.053


In [4]:
# Generate an instance of the plotter, add title and show latlon grid
gp = GroundtrackPlotter()
gp.update_layout(title="FOSSA-2E2 groundtrack")

# Plot previously defined EarthSatellite object
gp.plot(
    FOSSA_spacecraft,
    t_span,
    label="FOSSA-2E2",
    color="red",
    marker={
        "size": 10,
        "symbol": "triangle-right",
        "line": {"width": 1, "color": "black"},
    },
)
# For building geo traces
import plotly.graph_objects as go

# Position in [LAT LON]
STATION = [65.64737, -20.24609] * u.deg

# Let us add a new trace in original figure
gp.add_trace(
    go.Scattergeo(
        lat=STATION[0],
        lon=STATION[-1],
        name="Reyjkiavik",
        marker={"color": "blue"},
    )
)
gp.fig.show()